In [2]:
# 导入读取数据所需要的库
import numpy as np
import pandas as pd
from pandas import read_excel

import gensim
from gensim import models

df = read_excel(r'data\SEV_wordlist.xlsx')

In [3]:
# 预训练的词向量语料库文件路径
tecent_embedding_path = (r'D:\embedding_zh\tencent-ailab-embedding-zh-d200-v0.2.0-s\tencent-ailab-embedding-zh-d200-v0.2.0-s.txt')

# 加载词向量文件
tencent_embedding = models.KeyedVectors.load_word2vec_format(tecent_embedding_path, binary=False)

In [4]:
# 获取Excel文件中所有表的名称
excel_file = pd.ExcelFile(r'data\SEV_wordlist.xlsx')
print(excel_file.sheet_names)

['App_words', 'SES_words', 'Soc_words', 'Com_words', 'Mor_words']


In [5]:
# Mor扩展词表

df_Mor_words = pd.read_excel(r'data\SEV_wordlist.xlsx', sheet_name=4) 

In [6]:
# 求积极词汇的词表
df_Mor_positive = df_Mor_words.loc[df_Mor_words['Valence']=='Positive', 'Word']

# 格式转化为列表
Mor_positive_word_list = df_Mor_positive.tolist()

In [7]:
# 初始化有效种子词列表和相似词列表
Mor_positive_word_list_effective = []
Mor_similar_words_list = []

# 过滤词向量模型中存在的词并生成相似词
for word in Mor_positive_word_list:
    if word in tencent_embedding:
        Mor_positive_word_list_effective.append(word)
        # 找到与该词最相似的 10 个词
        Mor_similar_words = tencent_embedding.most_similar(word, topn=10)
        # 将相似词加入列表
        for similar_word, similarity in Mor_similar_words:
            Mor_similar_words_list.append(similar_word)
    else:
        print(f"Skipping {word}...")

Skipping 谦顺...
Skipping 恳挚...


In [8]:
Mor_unique_similar_words_list = list(set(Mor_similar_words_list))

df_unique_Mor_similar_words = pd.DataFrame(Mor_unique_similar_words_list, columns=['Word'])
print(df_unique_Mor_similar_words)

      Word
0       憨直
1     绝对公正
2     尽量节约
3     忠义两全
4     菩萨心肠
...    ...
1311  认真敬业
1312     勇
1313  自尊自爱
1314  言行合一
1315  英勇顽强

[1316 rows x 1 columns]


In [9]:
df_unique_Mor_similar_words.to_excel(r'result\\Mor_unique_positive_similar_words.xlsx', index=False)

In [10]:
# 消极词汇
df_Mor_negative = df_Mor_words.loc[df_Mor_words['Valence']=='Negative', 'Word']

# 格式转化为列表
Mor_negative_word_list = df_Mor_negative.tolist()

# 初始化有效种子词列表和相似词列表
Mor_negative_word_list_effective = []
Mor_similar_negative_words_list = []

# 过滤词向量模型中存在的词并生成相似词
for word in Mor_negative_word_list:
    if word in tencent_embedding:
        Mor_negative_word_list_effective.append(word)
        # 找到与该词最相似的 10 个词
        Mor_similar_negative_words = tencent_embedding.most_similar(word, topn=10)
        # 将相似词加入列表
        for similar_negative_word, similarity in Mor_similar_negative_words:
            Mor_similar_negative_words_list.append(similar_negative_word)
    else:
        print(f"Skipping {word}...")

Skipping 轻诺寡信...
Skipping 险诈...
Skipping 鼠肚鸡肠...
Skipping 耍心眼儿...
Skipping 害人虫...
Skipping 卑俗...


In [11]:
# 将相似词列表转换为集合再转换回列表，以去掉重复出现的词
Mor_unique_similar_negative_words_list = list(set(Mor_similar_negative_words_list))

# 将去重后的结果转换为 DataFrame
df_Mor_unique_similar_negative_words = pd.DataFrame(Mor_unique_similar_negative_words_list, columns=['Word'])
print(df_Mor_unique_similar_negative_words)

          Word
0         不顾大局
1         滥用民力
2        虚荣心作祟
3         过于狭隘
4           贪鄙
...        ...
1238      罪恶滔天
1239     蛮横不讲理
1240        淫虐
1241        独断
1242  吃着碗里看着锅里

[1243 rows x 1 columns]


In [12]:
# 保存去重后的相似词表到excel文件
df_Mor_unique_similar_negative_words.to_excel(r'result\\Mor_unique_negative_similar_words.xlsx', index=False)